# OCHA trigger thresholds

This notebook is for determining trigger thresholds for 
the OCHA Philippines trigger

In [115]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import genextreme as gev
from scipy.interpolate import interp1d

## Reading in the data

In [81]:
# Read in the typhoon info
filename = "../../IBF-Typhoon-model/data/loos_table_ibtrcs.csv"
df_typhoon = pd.read_csv(filename, index_col=0)
df_typhoon["year"] = df_typhoon["typhoon"].apply(lambda x: int(x[:4]))
df_typhoon

,Mun_Code,typhoon,No_Totally_DMG_BLD,year
0,PH175317000,1990129N05138,0,1990
1,PH175323000,1990129N05138,8,1990
2,PH012803000,1990163N08133,67,1990
3,PH012805000,1990163N08133,6,1990
4,PH012814000,1990163N08133,1,1990
...,...,...,...,...
106806,PH184531000,2022099N11127,4,2022
106807,PH184532000,2022099N11127,2,2022
106808,PH184608000,2022099N11127,7,2022
106809,PH184611000,2022099N11127,4,2022


In [82]:
# Read in the geo info
filename_geo = "../../IBF-Typhoon-model/data/gis_data/phl_admin3_simpl.geojson"
gdf = gpd.read_file(filename_geo)
gdf

,adm3_en,adm3_pcode,adm2_pcode,adm1_pcode,glat,glon,geometry
0,Adams,PH012801000,PH012800000,PH010000000,18.453,120.919,"MULTIPOLYGON (((120.94626 18.51158, 120.96915 ..."
1,Bacarra,PH012802000,PH012800000,PH010000000,18.267,120.613,"MULTIPOLYGON (((120.59081 18.29526, 120.59192 ..."
2,Badoc,PH012803000,PH012800000,PH010000000,17.906,120.509,"MULTIPOLYGON (((120.47783 17.97744, 120.47803 ..."
3,Bangui,PH012804000,PH012800000,PH010000000,18.490,120.750,"MULTIPOLYGON (((120.79393 18.53475, 120.79537 ..."
4,City of Batac,PH012805000,PH012800000,PH010000000,18.038,120.584,"MULTIPOLYGON (((120.56070 18.12259, 120.56088 ..."
...,...,...,...,...,...,...,...
1642,Tayasan,PH184622000,PH184600000,PH180000000,9.944,123.075,"MULTIPOLYGON (((123.16419 9.95278, 123.16998 9..."
1643,Valencia,PH184623000,PH184600000,PH180000000,9.291,123.161,"MULTIPOLYGON (((123.20764 9.32295, 123.21151 9..."
1644,Vallehermoso,PH184624000,PH184600000,PH180000000,10.343,123.293,"MULTIPOLYGON (((123.33592 10.38248, 123.33565 ..."
1645,Zamboanguita,PH184625000,PH184600000,PH180000000,9.162,123.162,"MULTIPOLYGON (((123.17078 9.22988, 123.18679 9..."


## Make a combined data frame with damage totals

In [90]:
# Merge them
df_comb = (pd.merge(df_typhoon, gdf[["adm3_pcode", "adm1_pcode"]], 
              left_on="Mun_Code", right_on="adm3_pcode", how="left")
      .drop(columns=["adm3_pcode"])
     )
df_comb

,Mun_Code,typhoon,No_Totally_DMG_BLD,year,adm1_pcode
0,PH175317000,1990129N05138,0,1990,PH170000000
1,PH175323000,1990129N05138,8,1990,PH170000000
2,PH012803000,1990163N08133,67,1990,PH010000000
3,PH012805000,1990163N08133,6,1990,PH010000000
4,PH012814000,1990163N08133,1,1990,PH010000000
...,...,...,...,...,...
106806,PH184531000,2022099N11127,4,2022,PH180000000
106807,PH184532000,2022099N11127,2,2022,PH180000000
106808,PH184608000,2022099N11127,7,2022,PH180000000
106809,PH184611000,2022099N11127,4,2022,PH180000000


In [101]:
# Group by admin1 and typhoon, and sum, then take max of each year
df_max = (df_comb.groupby(['adm1_pcode', 'typhoon'])
      .agg({'No_Totally_DMG_BLD': sum,'year': 'first'})
     .groupby(['adm1_pcode', 'year']).max()
     .rename(columns={'No_Totally_DMG_BLD': 'max_damage_event'})
     .reset_index()
             )
df_max

,adm1_pcode,year,max_damage_event
0,PH010000000,1990,2161
1,PH010000000,1991,4121
2,PH010000000,1992,5996
3,PH010000000,1993,7992
4,PH010000000,1994,1769
...,...,...,...
503,PH180000000,2018,542
504,PH180000000,2019,3381
505,PH180000000,2020,425
506,PH180000000,2021,145393


## Get the return periods

In [137]:
def get_rp_analytical(
    df_rp: pd.DataFrame,
    rp_var: str,
    show_plots: bool = False,
    plot_title: str = "",
    extend_factor: int = 1,
):
    """
    :param df_rp: DataFrame where the index is the year, and the rp_var
    column contains the maximum value per year
    :param rp_var: The column with the quantity to be evaluated
    :param show_plots: Show the histogram with GEV distribution overlaid
    :param plot_title: The title of the plot
    :param extend_factor: Extend the interpolation range in case you want to
    calculate a relatively high return period
    :return: Interpolated function that gives the quantity for a
    given return period
    """
    df_rp = df_rp.sort_values(by=rp_var, ascending=False)
    rp_var_values = df_rp[rp_var]
    shape, loc, scale = gev.fit(
        rp_var_values,
        loc=rp_var_values.median(),
        scale=rp_var_values.median() / 2,
    )
    x = np.linspace(
        rp_var_values.min(),
        rp_var_values.max() * extend_factor,
        100 * extend_factor,
    )
    if show_plots:
        fig, ax = plt.subplots()
        ax.hist(rp_var_values, density=True, bins=20)
        ax.plot(x, gev.pdf(x, shape, loc, scale))
        ax.set_title(plot_title)
        plt.show()
    y = gev.cdf(x, shape, loc, scale)
    y = 1 / (1 - y)
    return interp1d(y, x)


def get_rp_empirical(df_rp: pd.DataFrame, rp_var: str):
    """
    :param df_rp: DataFrame where the index is the year, and the rp_var
    column contains the maximum value per year
    :param rp_var: The column
    with the quantity to be evaluated
    :return: Interpolated function
    that gives the quantity for a give return period
    """
    df_rp = df_rp.sort_values(by=rp_var, ascending=False)
    n = len(df_rp)
    df_rp["rank"] = np.arange(n) + 1
    df_rp["exceedance_probability"] = df_rp["rank"] / (n + 1)
    df_rp["rp"] = 1 / df_rp["exceedance_probability"]
    return interp1d(df_rp["rp"], df_rp[rp_var])



def get_rp_df(
    df: pd.DataFrame,
    rp_var: str,
    years: list = None,
    method: str = "analytical",
    show_plots: bool = False,
    extend_factor: int = 1,
    round_rp: bool = True,
) -> pd.DataFrame:
    """
    Function to get the return periods, either empirically or
    analytically See the `glofas/utils.py` to do this with a xarray
    dataset instead of a dataframe
    :param df: Dataframe with data to compute rp on
    :param rp_var: column name to compute return period on
    :param years: Return period years to compute
    :param method: Either "analytical" or "empirical"
    :param show_plots: If method is analytical, can show the histogram and GEV
    distribution overlaid
    :param extend_factor: If method is analytical, can extend the interpolation
    range to reach higher return periods
    :param round_rp: if True, round the rp values, else return original values
    :return: Dataframe with return period years as index and stations as
    columns
    """
    if years is None:
        years = [1.5, 2, 3, 5]
    df_rps = pd.DataFrame(columns=["rp"], index=years)
    if method == "analytical":
        f_rp = get_rp_analytical(
            df_rp=df,
            rp_var=rp_var,
            show_plots=show_plots,
            extend_factor=extend_factor,
        )
    elif method == "empirical":
        f_rp = get_rp_empirical(
            df_rp=df,
            rp_var=rp_var,
        )
    else:
        logger.error(f"{method} is not a valid keyword for method")
        return None
    df_rps["rp"] = f_rp(years)
    if round_rp:
        df_rps["rp"] = np.round(df_rps["rp"])
    return df_rps


In [151]:
# Only do for regions 5 and 8
ROI = {
    "5": ["PH050000000"], 
    "8": ["PH080000000"], 
    "both": ["PH050000000", "PH080000000"]
}
years = [1.5, 2, 5, 10, 20]
df_rp = pd.DataFrame()
for region_key, region_value in ROI.items():
    df = df_max.loc[df_max["adm1_pcode"].isin(region_value)]
    if len(region_value) > 1:
         df = df.groupby("year").sum().reset_index()
    df = df.set_index(df.year)
    df_rp_tmp = (get_rp_df(df, "max_damage_event", years=years, extend_factor=2,
                          method="analytical")
                 .reset_index()
                 .rename(columns={"rp": "max_damage", "index": "rp"})
                )
    df_rp_tmp["region"] = region_key
    df_rp = pd.concat([df_rp, df_rp_tmp], ignore_index=True)

df_rp


,rp,max_damage,region
0,1.5,7492.0,5
1,2.0,13056.0,5
2,5.0,41739.0,5
3,10.0,84458.0,5
4,20.0,162911.0,5
5,1.5,3307.0,8
6,2.0,6751.0,8
7,5.0,35094.0,8
8,10.0,101588.0,8
9,20.0,279956.0,8
